This notebook shows the effect of noise on RefCM's performance in matching datasets.

In [1]:
import sys
sys.path.append('../src/')

import numpy as np
import scanpy as sc
import config
import logging
import plotly.express as px

from refcm import RefCM
from anndata import AnnData

config.start_logging(logging.INFO)

For this particular example, we will utilize the Allen Brain datasets.

In [2]:
dss = {
    'MTG': sc.read_h5ad('../data/MTG.h5ad'),
    'ALM': sc.read_h5ad('../data/ALM.h5ad'),
    'VISp': sc.read_h5ad('../data/VISp.h5ad')
}

There are two main types of noise we will consider:
* measurement noise, i.e. noise in the raw gene expression counts.
* clustering noise, i.e. cell types not being clustered together correctly.

Let's first look at the effect of adding measurement noise to our data. We will define the following function to simulate this effect.

In [3]:
def add_measurement_noise(ds: AnnData, epsilon: float) -> AnnData:
    """
    Adds epsilon * std_dev of noise to each cell in the dataset.
    Saves the original counts under .layers['original'].

    Parameters
    ----------
    ds: AnnData
        The AnnData dataset to add noise to. Assumes .X is raw counts.
    epsilon: float
        epsilon * std_dev of noise to add to each cell's raw counts.
        
    Returns
    -------
    AnnData:
        The original dataset, with the added noise.
    """
    ds.layers['original'] = ds.X.copy()
    
    stds = ds.X.std(axis=1)
    noise = np.zeros_like(ds.X)
    for i, std in enumerate(stds):
        noise[i] = np.random.normal(0, epsilon * std, size=noise.shape[1])
        
    ds.X = (ds.X + noise).clip(0).round()
    
    return ds

def undo_measurement_noise(ds: AnnData) -> AnnData:
    """ Undoes the effects of the previous function."""
    ds.X = ds.layers.pop('original')
    return ds

We can then look at the impact of applying different quantities of noise on all different combinations of Allen-Brain dataset matching tests as follows. In this particular case, we will choosing to add the measurement noise to the query dataset.

In [4]:
rcm = RefCM(cache_load=False, cache_save=False)

epsilons = [round(e, 2) for e in np.linspace(0, 5, 26)]
combinations = [(q, ref) for q in dss for ref in dss if q != ref]
results = np.zeros((len(combinations), len(epsilons)))

for i, (q, ref) in enumerate(combinations):    
    
    for j, epsilon in enumerate(epsilons):
        
        # add noise and run refcm
        add_measurement_noise(dss[q], epsilon)
        m = rcm.annotate(dss[q], q, dss[ref], ref, 'labels34', 'labels34')
        m.eval(ground_truth_obs_key='labels34')
        
        # retrieve the number of correct clusters mapped
        results[i, j] = m.n_correct
        
        # undo measurement noise and report results
        undo_measurement_noise(dss[q])
        print(f'     Noise: {epsilon:.2f} | {q:>4}->{ref:<4} : {int(results[i, j]):<2}/34')
        

[refcm           ] [INFO    ] : NOTE: raw counts expected in anndata .X attributes.
|████████████████| [100.00% ] : 00:34
[matchings       ] [INFO    ] : mapped MTG                  to ALM                 
[matchings       ] [INFO    ] : (34 common cell types)
[matchings       ] [INFO    ] : 34/34 correct mappings
[matchings       ] [INFO    ] : 0 /34 incorrect mappings


     Noise: 0.00 |  MTG->ALM  : 34/34


|████████████████| [100.00% ] : 00:34
[matchings       ] [INFO    ] : mapped MTG                  to ALM                 
[matchings       ] [INFO    ] : (34 common cell types)
[matchings       ] [INFO    ] : 34/34 correct mappings
[matchings       ] [INFO    ] : 0 /34 incorrect mappings


     Noise: 0.20 |  MTG->ALM  : 34/34


|████████████████| [100.00% ] : 00:30
[matchings       ] [INFO    ] : mapped MTG                  to ALM                 
[matchings       ] [INFO    ] : (34 common cell types)
[matchings       ] [INFO    ] : 34/34 correct mappings
[matchings       ] [INFO    ] : 0 /34 incorrect mappings


...

     Noise: 4.60 | VISp->ALM  : 32/34


|████████████████| [100.00% ] : 00:28
[matchings       ] [INFO    ] : mapped VISp                 to ALM                 
[matchings       ] [INFO    ] : (34 common cell types)
[matchings       ] [INFO    ] : 32/34 correct mappings
[matchings       ] [INFO    ] : 2 /34 incorrect mappings


     Noise: 4.80 | VISp->ALM  : 32/34


|████████████████| [100.00% ] : 00:28
[matchings       ] [INFO    ] : mapped VISp                 to ALM                 
[matchings       ] [INFO    ] : (34 common cell types)
[matchings       ] [INFO    ] : 32/34 correct mappings
[matchings       ] [INFO    ] : 2 /34 incorrect mappings


     Noise: 5.00 | VISp->ALM  : 32/34


In [5]:
fig = px.imshow(
    results, 
    title='Robustness to gene expression noise', 
    color_continuous_scale='Sunset',
    labels=dict(x="Noise (std devs)", y="Query->Ref", color="correct clusters (/34)"),
    x=epsilons,
    y=[' -> '.join(t) for t in combinations]
)
fig.update_xaxes(dtick=0.2)
fig.show()

These tests can be easily modified to observe the effects of adding noise to the reference datasets instead, by altering the following lines inside the loop:
```python
    add_measurement_noise(dss[ref], epsilon)
    ...
    undo_measurement_noise(dss[ref])
```

Now we can analyze the effects of clustering noise, where we assume that the clustering step did not fully separate what should be different cell types. Here we will simulate this effect on the reference dataset, although it can of course be easily mimicked to apply to the query dataset instead.

In [6]:
def add_clustering_noise(ds: AnnData, cluster_key: str, epsilon: float) -> AnnData:
    """
    Adds clustering noise whereby epsilon % of each cluster is 
    randomly re-assigned to other clusters. 
    The original clusters are saved under .obs['original']

    Parameters
    ----------
    ds: AnnData
        The AnnData dataset to add noise to.
    cluster_key: str
        The .obs key for the clusters to add noise to.
    epsilon: float
        percentage of cells per cluster to re-assign
    
    Returns
    -------
    AnnData:
        The original dataset, with the added clustering noise.
    """
    ds.obs['original'] = ds.obs[cluster_key].copy()
    clusters = ds.obs['original'].unique().tolist()
        
    for cluster in clusters:
        c_mask = np.argwhere(ds.obs['original'] == cluster).squeeze()
        n_relabel = int(epsilon * len(c_mask))
        
        chosen = np.random.choice(c_mask, n_relabel, replace=False)
        dest_clusters = np.random.choice([c for c in clusters if c != 'SSt 1'], n_relabel)
        
        ds.obs[cluster_key][chosen] = dest_clusters
    
    return ds

def undo_clustering_noise(ds: AnnData, cluster_key: str) -> AnnData:
    """ Undoes the effects of the previous function."""
    ds.obs[cluster_key] = ds.obs['original'].copy()
    ds.obs = ds.obs.drop('original', axis=1)
    return ds

As before, we can now utilize this to visualize the impact of applying different quantities of noise on all different combinations of Allen-Brain dataset pair-matching:

In [7]:
rcm = RefCM(cache_load=False, cache_save=False)

epsilons = [round(e, 4) for e in np.linspace(0, 0.5, 21)]
combinations = [(q, ref) for q in dss for ref in dss if q != ref]
results = np.zeros((len(combinations), len(epsilons)))

for i, (q, ref) in enumerate(combinations):    
    
    for j, epsilon in enumerate(epsilons):
        
        # add clustering noise and run refcm
        add_clustering_noise(dss[ref], 'labels34', epsilon)
        m = rcm.annotate(dss[q], q, dss[ref], ref, 'labels34', 'labels34')
        m.eval(ground_truth_obs_key='labels34')
        
        # retrieve the number of correct clusters mapped
        results[i, j] = m.n_correct
        
        # undo clustering noise and report results
        undo_clustering_noise(dss[ref], 'labels34')
        print(f'   Noise: {epsilon*100:>5.2f}% | {q:>4}->{ref:<4} : {int(results[i, j]):<2}/34')

[refcm           ] [INFO    ] : NOTE: raw counts expected in anndata .X attributes.
|████████████████| [100.00% ] : 00:32
[matchings       ] [INFO    ] : mapped MTG                  to ALM                 
[matchings       ] [INFO    ] : (34 common cell types)
[matchings       ] [INFO    ] : 34/34 correct mappings
[matchings       ] [INFO    ] : 0 /34 incorrect mappings


   Noise:  0.00% |  MTG->ALM  : 34/34


|████████████████| [100.00% ] : 00:32
[matchings       ] [INFO    ] : mapped MTG                  to ALM                 
[matchings       ] [INFO    ] : (34 common cell types)
[matchings       ] [INFO    ] : 32/34 correct mappings
[matchings       ] [INFO    ] : 2 /34 incorrect mappings


   Noise:  2.50% |  MTG->ALM  : 32/34


|████████████████| [100.00% ] : 00:33
[matchings       ] [INFO    ] : mapped MTG                  to ALM                 
[matchings       ] [INFO    ] : (34 common cell types)
[matchings       ] [INFO    ] : 32/34 correct mappings
[matchings       ] [INFO    ] : 2 /34 incorrect mappings


...

   Noise: 45.00% | VISp->ALM  : 32/34


|████████████████| [100.00% ] : 00:26
[matchings       ] [INFO    ] : mapped VISp                 to ALM                 
[matchings       ] [INFO    ] : (34 common cell types)
[matchings       ] [INFO    ] : 32/34 correct mappings
[matchings       ] [INFO    ] : 2 /34 incorrect mappings


   Noise: 47.50% | VISp->ALM  : 32/34


|████████████████| [100.00% ] : 00:25
[matchings       ] [INFO    ] : mapped VISp                 to ALM                 
[matchings       ] [INFO    ] : (34 common cell types)
[matchings       ] [INFO    ] : 32/34 correct mappings
[matchings       ] [INFO    ] : 2 /34 incorrect mappings


   Noise: 50.00% | VISp->ALM  : 32/34


In [8]:
fig = px.imshow(
    results, 
    title='Robustness to clustering noise', 
    color_continuous_scale='Sunset',
    labels=dict(x="Noise (%)", y="Query -> Ref", color="correct clusters (/34)"),
    x=epsilons,
    y=[' -> '.join(t) for t in combinations]
)
fig.update_xaxes(dtick=0.2)
fig.show()